In [ ]:
# !pip install -U "langchain==0.0.344" openai tiktoken lark datasets sentence_transformers FlagEmbedding lancedb -qq


In [2]:
from langchain.vectorstores import LanceDB
from langchain.retrievers import ParentDocumentRetriever

#Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.docstore.document import Document

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

import os
from datasets import load_dataset

from langchain.embeddings import HuggingFaceBgeEmbeddings
import lancedb
from dotenv import dotenv_values
import openai

env_vars = dotenv_values('.env')
openai.api_key = "sk-proj-Tzc9mrWyEFVxyDsq5HiWT3BlbkFJxp47toOztG4XRILBeRxr"

# Embedding Functions
# model_name = "BAAI/bge-small-en-v1.5" # Open Source and effective Embedding
# encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
# bge_embeddings = HuggingFaceBgeEmbeddings(model_name=model_name,model_kwargs={'device': 'cuda'},encode_kwargs=encode_kwargs)
bge_embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai.api_key)
# Data Chunking Functions
small_chunk_splitter = RecursiveCharacterTextSplitter(chunk_size = 512) # Splitter to split documents into small chunks
big_chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=2048) # Another Level of Bigger Chunks

# Lance DB Connection. Load if exists else create
my_db = lancedb.connect("./my_db")

In [11]:
print(openai.api_key)

sk-proj-Tzc9mrWyEFVxyDsq5HiWT3BlbkFJxp47toOztG4XRILBeRxr


In [ ]:
# Load a sample data here
long_texts = load_dataset("huggingartists/eminem")["train"].to_pandas().sample(100)["text"] # Data of huge context length. Use 100 random examples for demo

# Convert to LangChain Document object
docs = [Document(page_content=content, doc_id = _id, metadata = {"doc_id":_id}) for (_id, content) in enumerate(long_texts)] # List of LangChain Document Objects


# if "small_chunk_table" in my_db.table_names():
#   small_chunk_table = my_db.open_table("small_chunk_table")
# else: # NOTE: 384 is the size of BAAI Embedding and -999 because it's a dummy data so invalid Embedding
small_chunk_table = my_db.create_table("small_chunk_table", data=[{"vector": [], "text": "", "doc_id": "-1",}], mode="overwrite")

# small_chunk_table.delete('doc_id = "-1"')

vectorstore = LanceDB(small_chunk_table, bge_embeddings) # Vectorstore to use to index the child chunks
store = InMemoryStore() # The storage layer for the parent documents

full_doc_retriever = ParentDocumentRetriever(vectorstore=vectorstore, docstore=store, child_splitter=small_chunk_splitter)

full_doc_retriever.add_documents(docs, ids=None) # Add all the documents

In [ ]:
print(len(bge_embeddings.embed_query("test")))

In [ ]:
# Fetch 3 most similar Smaller Documents
sub_docs = vectorstore.similarity_search("I am whatever you say I am and if I wasn't why would you say I am", k=3)

print(sub_docs[0].page_content) # This is a Smaller Chunk.


full_docs = full_doc_retriever.get_relevant_documents("I am whatever you say I am and if I wasn't why would you say I am", k = 3)
print(full_docs[0].page_content) # This is the Parent Document returned after matching the smaller chunks internally

# Bigger Chunk Retrieval

In [ ]:
if "big_chunk_table" in my_db.table_names():
  big_chunk_table = my_db.open_table("big_chunk_table")
else:
  big_chunk_table = my_db.create_table("big_chunk_table", data=[{"vector": [-999]*384, "text": "", "doc_id": "-1",}], mode="overwrite")

big_chunk_table.delete('doc_id = "-1"')

vectorstore = LanceDB(big_chunk_table, bge_embeddings)
store = InMemoryStore()

big_chunk_retriever = ParentDocumentRetriever(vectorstore=vectorstore, docstore=store,
                                              child_splitter = small_chunk_splitter,
                                              parent_splitter = big_chunk_splitter) # See one more line addition which retrieves the larger chunk instead of Parent Document

big_chunk_retriever.add_documents(docs, ids=None) # Add all the documents


big_chunks_docs = big_chunk_retriever.get_relevant_documents("I am whatever you say I am and if I wasn't why would you say I am", k = 3)
print(big_chunks_docs[0].page_content) # This is the BIG chunks (in place of Parent Document) returned after matching the smaller chunks internally